In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|default_exp agents.discrete

In [ ]:
#|export
# Python native modules
import os
from typing import Union
# Third party libs
import torchdata.datapipes as dp
import torch
# from torch.nn import *
import torch.nn.functional as F
from torchdata.dataloader2.graph import traverse_dps
import numpy as np
# Local modules
from fastrl.pipes.core import find_dp
from fastrl.agents.core import AgentBase
from fastrl.loggers.core import Record,is_record,not_record

# Agent Discrete
> DataPipes used by Agent operating in the Discrete action space.

In [ ]:
#|export
class ArgMaxer(dp.iter.IterDataPipe):
    "Given input `Tensor` from `source_datapipe` returns a tensor of same shape with argmax set to 1."
    def __init__(self,source_datapipe,axis=1,only_idx=False): 
        self.source_datapipe = source_datapipe
        self.axis = axis
        self.only_idx = only_idx
        
    def debug_display(self,step,idx):
        print(f'Step: {step}\n{idx}')
    
    def __iter__(self) -> torch.LongTensor:
        for step in self.source_datapipe:
            if not issubclass(step.__class__,torch.Tensor):
                raise Exception(f'Expected Tensor to take the argmax, got {type(step)}\n{step}')
            # Might want to support simple tuples also depending on if we are processing multiple fields.
            idx = torch.argmax(step,axis=self.axis).reshape(-1,1)
            if self.only_idx: 
                yield idx.long()
                continue
            step[:] = 0
            step.scatter_(1,idx,1)
            yield step.long()
            

In [ ]:
from fastrl.agents.dqn.basic import DQN
from fastrl.agents.core import StepFieldSelector,SimpleModelRunner,AgentBase,AgentHead
from fastrl.core import SimpleStep

In [ ]:
# from fastrl.agents.dqn.basic import DQN
# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2)
# Setup the agent
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
agent = AgentHead(agent)

for action in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]))]):
    print(action)

traverse_dps(agent)

In [ ]:
#|export
class EpsilonSelector(dp.iter.IterDataPipe):
    debug=False
    "Given input `Tensor` from `source_datapipe`."
    def __init__(self,
            source_datapipe, # a datapipe whose next(source_datapipe) -> `Tensor` 
            min_epsilon:float=0.2, # The minimum epsilon to drop to
            # The max/starting epsilon if `epsilon` is None and used for calculating epislon decrease speed.
            max_epsilon:float=1, 
            # Determines how fast the episilon should drop to `min_epsilon`. This should be the number
            # of steps that the agent was run through.
            max_steps:int=100,
            # The starting epsilon
            epsilon:float=None,
            # Based on the `base_agent.model.training`, by default no decrement or step tracking will
            # occur during validation steps.
            decrement_on_val:bool=False,
            # Based on the `base_agent.model.training`, by default random actions will not be attempted
            select_on_val:bool=False,
            # Also return the mask that, where True, the action should be randomly selected.
            ret_mask:bool=False,
            # The device to create the masks one
            device='cpu'
        ): 
        self.source_datapipe = source_datapipe
        self.min_epsilon = min_epsilon
        self.max_epsilon = max_epsilon
        self.max_steps = max_steps
        self.epsilon = epsilon
        self.decrement_on_val = decrement_on_val
        self.select_on_val = select_on_val
        self.ret_mask = ret_mask
        self.agent_base = find_dp(traverse_dps(self.source_datapipe),AgentBase)
        self.step = 0
        self.device = torch.device(device)
    
    def __iter__(self):
        for action in self.source_datapipe:
            # TODO: Support tuples of actions also
            if not issubclass(action.__class__,torch.Tensor):
                raise Exception(f'Expected Tensor, got {type(action)}\n{action}')
            if action.dtype!=torch.int64:
                raise ValueError(f'Expected Tensor of dtype int64, got: {action.dtype} from {self.source_datapipe}')
                
            if self.agent_base.model.training or self.decrement_on_val:
                self.step+=1
                
            self.epsilon = max(self.min_epsilon,self.max_epsilon-self.step/self.max_steps)
            # Add a batch dim if missing
            if len(action.shape)==1: action.unsqueeze_(0)
            mask = None
            if self.agent_base.model.training or self.select_on_val:
                # Given N(action.shape[0]) actions, select the ones we want to randomly assign... 
                mask = torch.rand(action.shape[0],).to(self.device)<self.epsilon
                # Get random actions as their indexes
                rand_action_idxs = torch.LongTensor(int(mask.sum().long()),).to(self.device).random_(action.shape[1])
                # If the input action is [[0,1],[1,0]] and...
                # If mask is [True,False] and...
                # if rand_action_idxs is [0]
                # the action[mask] will have [[1,0]] assigned to it resulting in... 
                # an action with [[1,0],[1,0]]
                # print(action.shape[1])
                # if self.debug: print(f'Mask: {mask}\nRandom Actions: {rand_action_idxs}\nPre-random Actions: {action}')
                action[mask] = F.one_hot(rand_action_idxs,action.shape[1])
            
            yield ((action,mask) if self.ret_mask else action)

Check that when `min_epsilon=1`, that the actions have 100% likihood of randomness applied 
(even though some might not change due to the random action matching the chosen action). Check that this 
works on a large batch of `200 steps`...

In [ ]:
from fastcore.all import test_eq,test_ne

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
selector = EpsilonSelector(agent,min_epsilon=1,ret_mask=True)
agent = AgentHead(selector)

for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]*200))]):
    test_eq(mask.sum(),200)
    test_ne(action.sum(0)[0],200) # Only some of the actions should 1
    test_ne(action.sum(0)[1],0) # Only some of the actions should be 0
    test_eq(selector.epsilon,1)
    test_eq(selector.step,1)

traverse_dps(agent)

Check that when `min_epsilon=1`, that the actions have 100% likihood of randomness applied 
(even though some might not change due to the random action matching the chosen action). Check that this 
works on single batches over `200 steps`...

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent,axis=1)
selector = EpsilonSelector(agent,min_epsilon=1,ret_mask=True)
agent = AgentHead(selector)

actions = None
for i in range(200):
    for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]))]):
        test_eq(mask.sum(),1)
        if actions is None: actions = action
        else:               actions = torch.vstack((actions,action))
test_ne(actions.sum(0)[0],200) # Only some of the actions should 1
test_ne(actions.sum(0)[1],0) # Only some of the actions should be 0
test_eq(selector.epsilon,1)
test_eq(selector.step,200)
traverse_dps(agent)

Check that when `min_epsilon=0 and max_epsilon=0`, that the actions have 0% likihood of randomness applied. Check that this 
works on a large batch of `200 steps`...

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
selector = EpsilonSelector(agent,min_epsilon=0,max_epsilon=0,ret_mask=True)
agent = AgentHead(selector)

for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]*200))]):
    test_eq(mask.sum(),0)
    test_eq(action.sum(0)[0],200) # All the "left" actions should be 1
    test_eq(action.sum(0)[1],0) # All the "right" actions should be 0
    test_eq(selector.epsilon,0)
    test_eq(selector.step,1)
traverse_dps(agent)

Check that when `min_epsilon=0 and max_epsilon=0`, that the actions have 0% likihood of randomness applied. Check that this 
works on single batches over `200 steps`...

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
selector = EpsilonSelector(agent,min_epsilon=0,max_epsilon=0,ret_mask=True)
agent = AgentHead(selector)

actions = None
for i in range(200):
    for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]))]):
        test_eq(mask.sum(),0)
        if actions is None: actions = action
        else:               actions = torch.vstack((actions,action))
test_eq(actions.sum(0)[0],200) # All the "left" actions should be 1
test_eq(actions.sum(0)[1],0) # All the "right" actions should be 0
test_eq(selector.epsilon,0)
test_eq(selector.step,200)
traverse_dps(agent)

Check that when `min_epsilon=0 and max_epsilon=1`, the actions should become less random
as the steps go on. Check that this works on a large batch of `200 steps`...

`epislon` should be 0 at the end of this...

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
selector = EpsilonSelector(agent,min_epsilon=0,max_epsilon=1,max_steps=100,ret_mask=True)
agent = AgentHead(selector)

actions = None
masks = None
epsilons = None
for i in range(200):
    for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]*200))]):
        if actions is None: actions = action
        else:               actions = torch.vstack((actions,action))
        if masks is None: masks = mask
        else:             masks = torch.hstack((masks,mask))
        if epsilons is None: epsilons = torch.tensor([selector.epsilon])
        else:                epsilons = torch.hstack((epsilons,torch.tensor([selector.epsilon])))
        
test_ne(masks[:((200*200)//2)].sum(),200) # We do not expect this to equal a perfect 200...
test_ne(masks[:((200*200)//2)].sum(),0)   # ... but we also dont expect it to be 0
assert 1000<masks[:((200*200)//2)].sum()<10_000,\
        """We expect this to be somewhere between 1000 and 10,000, generally in the 9000 range since 
           for 200 steps, we are running 200 inputs"""
test_eq(masks[((200*200)//2):].sum(),0) # We fully expect this to be 0 after the half way point
test_ne(actions.sum(0)[0],200) # All the "left" generally shouldnt be 1
test_ne(actions.sum(0)[1],0) # All the "right"  generally shouldnt be 0
test_eq(selector.epsilon,0)
test_eq(selector.step,200)
# Since the max steps are 100, and we go for 200 steps, the first 100 epislon entries shouldnt be 0
test_ne(epsilons[:100].sum(),0) 
# In fact the first 100 should sum up to somewhere between 40 and 50. (expected 49.5)
test_eq(40<epsilons[:100].sum()<50,True) 
# Everything after 100 should be 0
test_eq(epsilons[100:].sum(),0)
traverse_dps(agent)

Check that when `min_epsilon=0 and max_epsilon=0`, that the actions have 0% likihood of randomness applied. Check that this 
works on single batches over `200 steps`...

In [ ]:
agent = AgentBase(model)
agent = StepFieldSelector(agent,field='state')
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent)
selector = EpsilonSelector(agent,min_epsilon=0,max_epsilon=1,max_steps=100,ret_mask=True)
agent = AgentHead(selector)

actions = None
masks = None
epsilons = None
for i in range(200):
    for action,mask in agent([SimpleStep.random(state=torch.tensor([[1.,2.,3.,4.]]))]):
        if actions is None: actions = action
        else:               actions = torch.vstack((actions,action))
        if masks is None: masks = mask
        else:             masks = torch.hstack((masks,mask))
        if epsilons is None: epsilons = torch.tensor([selector.epsilon])
        else:                epsilons = torch.hstack((epsilons,torch.tensor([selector.epsilon])))
        
test_ne(masks[:(200//2)].sum(),200) # We do not expect this to equal a perfect 200...
test_ne(masks[:(200//2)].sum(),0)   # ... but we also dont expect it to be 0
assert 40<masks[:(200//2)].sum()<60,'We expect this to be somewhere between 60, generally in the ~50 range'
test_eq(masks[(200//2):].sum(),0) # We fully expect this to be 0 after the half way point
test_ne(actions.sum(0)[0],200) # All the "left" generally shouldnt be 1
test_ne(actions.sum(0)[1],0) # All the "right"  generally shouldnt be 0
test_eq(selector.epsilon,0)
test_eq(selector.step,200)
# Since the max steps are 100, and we go for 200 steps, the first 100 epislon entries shouldnt be 0
test_ne(epsilons[:100].sum(),0) 
# In fact the first 100 should sum up to somewhere between 40 and 50. (expected 49.5)
test_eq(40<epsilons[:100].sum()<50,True) 
# Everything after 100 should be 0
test_eq(epsilons[100:].sum(),0)

traverse_dps(agent)

In [ ]:
#|export
class EpsilonCollector(dp.iter.IterDataPipe):
    title:str='epsilon'
    
    def __init__(self,
         source_datapipe # The parent datapipe, likely the one to collect metrics from
        ):
        self.source_datapipe = source_datapipe
        
    def __iter__(self):
        # for q in self.main_buffers: q.append(Record('epsilon',None))
        yield Record(self.title,None)
        for action in self.source_datapipe:
            if not_record(action):
                yield Record(self.title,self.source_datapipe.epsilon)
            yield action

In [ ]:
#|export
class PyPrimativeConverter(dp.iter.IterDataPipe):
    debug=False
    
    "Given input `Tensor` from `source_datapipe` returns a numpy array of same shape with argmax set to 1."
    def __init__(self,source_datapipe,remove_batch_dim=True): 
        self.source_datapipe = source_datapipe
        self.remove_batch_dim = remove_batch_dim
        
    def debug_display(self,step): print(f'Step: {step}')
    
    def __iter__(self) -> Union[float,bool,int]:
        for step in self.source_datapipe:
            if not issubclass(step.__class__,(np.ndarray)):
                raise Exception(f'Expected list or np.ndarray to  convert to python primitive, got {type(step)}\n{step}')
            
            if len(step)>1 or len(step)==0:
                raise Exception(f'`step` from {self.source_datapipe} needs to be len 1, not {len(step)}')
            else:
                step = step[0]
                
            if np.issubdtype(step.dtype,np.integer):
                yield int(step)
            elif np.issubdtype(step.dtype,np.floating):
                yield float(step)
            elif np.issubdtype(step.dtype,np.bool8):
                yield bool(step)
            else:
                raise Exception(f'`step` from {self.source_datapipe} must be one of the 3 python types: bool,int,float, not {step.dtype}')

In [ ]:
pipe = PyPrimativeConverter([np.array([0.5])])
list(pipe)

In [ ]:
pipe = PyPrimativeConverter([np.array([1])])
list(pipe)

In [ ]:
pipe = PyPrimativeConverter([np.array([True])])
list(pipe)

In [ ]:
# # Setup up the core NN
# torch.manual_seed(0)
# model = DQN(4,2)

# agent = AgentBase(model)
# agent = LoggerBasePassThrough(agent,[logger_base])
# agent = StepFieldSelector(agent,field='state')
# agent = SimpleModelRunner(agent)
# agent = ArgMaxer(agent)
# selector = EpsilonSelector(agent,min_epsilon=0,max_epsilon=1,max_steps=100)
# agent = EpsilonCollector(selector)
# agent = ArgMaxer(agent,only_idx=True)
# agent = NumpyConverter(agent)
# agent = PyPrimativeConverter(agent)
# agent = AgentHead(agent)

# traverse(agent)

In [ ]:
# for action in agent([SimpleStep.random(state=tensor([[1.,2.,3.,4.]]))]*800):
#     pass # print(action)

# epsilon_logs = list(logger_base.dequeue())
# test_eq(len(epsilon_logs),801)

In [ ]:
#|hide
#|eval: false
!nbdev_export